In [1]:
import os
import tensorflow as tf
from d2l import tensorflow as d2l

# Sequence to sequence model

In [2]:
import numpy as np
from tensorflow import keras

In [3]:
!!curl -O http://www.manythings.org/anki/spa-eng.zip
!!unzip spa-eng.zip  

['Archive:  spa-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

## Preparing the dataset

In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "spa.txt"

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 69
Number of unique output tokens: 84
Max sequence length for inputs: 16
Max sequence length for outputs: 78


In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [8]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


## Building the Encoder-Decoder

In [9]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Training

In [12]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [13]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s-esp")


Epoch 1/100
125/125 [==============================] - 29s 208ms/step - loss: 1.1221 - accuracy: 0.7808 - val_loss: 0.7266 - val_accuracy: 0.8007
Epoch 2/100
125/125 [==============================] - 24s 194ms/step - loss: 0.6127 - accuracy: 0.8339 - val_loss: 0.5997 - val_accuracy: 0.8323
Epoch 3/100
125/125 [==============================] - 26s 210ms/step - loss: 0.5057 - accuracy: 0.8615 - val_loss: 0.5179 - val_accuracy: 0.8500
Epoch 4/100
125/125 [==============================] - 29s 233ms/step - loss: 0.4350 - accuracy: 0.8742 - val_loss: 0.4810 - val_accuracy: 0.8563
Epoch 5/100
125/125 [==============================] - 35s 279ms/step - loss: 0.3937 - accuracy: 0.8823 - val_loss: 0.4537 - val_accuracy: 0.8625
Epoch 6/100
125/125 [==============================] - 31s 244ms/step - loss: 0.3662 - accuracy: 0.8900 - val_loss: 0.4356 - val_accuracy: 0.8677
Epoch 7/100
125/125 [==============================] - 37s 294ms/step - loss: 0.3472 - accuracy: 0.8950 - val_loss: 0.4207 -

Epoch 57/100
125/125 [==============================] - 25s 199ms/step - loss: 0.0530 - accuracy: 0.9844 - val_loss: 0.5098 - val_accuracy: 0.8960
Epoch 58/100
125/125 [==============================] - 24s 196ms/step - loss: 0.0519 - accuracy: 0.9844 - val_loss: 0.5168 - val_accuracy: 0.8964
Epoch 59/100
125/125 [==============================] - 26s 208ms/step - loss: 0.0498 - accuracy: 0.9851 - val_loss: 0.5174 - val_accuracy: 0.8969
Epoch 60/100
125/125 [==============================] - 25s 203ms/step - loss: 0.0481 - accuracy: 0.9855 - val_loss: 0.5229 - val_accuracy: 0.8961
Epoch 61/100
125/125 [==============================] - 25s 197ms/step - loss: 0.0470 - accuracy: 0.9859 - val_loss: 0.5291 - val_accuracy: 0.8965
Epoch 62/100
125/125 [==============================] - 25s 200ms/step - loss: 0.0456 - accuracy: 0.9863 - val_loss: 0.5318 - val_accuracy: 0.8964
Epoch 63/100
125/125 [==============================] - 26s 204ms/step - loss: 0.0441 - accuracy: 0.9871 - val_loss: 0

INFO:tensorflow:Assets written to: s2s-esp/assets


INFO:tensorflow:Assets written to: s2s-esp/assets


## Prediction

In [14]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s-esp")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

In [15]:
decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]

In [16]:
# We keep the states of decoder LSTM for inference
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)

decoder_states = [state_h_dec, state_c_dec]

In [17]:
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [18]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0
  
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [33]:
_, unique_strings = np.unique(input_texts, return_index=True)

In [34]:
for seq_index in unique_strings[:50]:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: A bird can fly.
Decoded sentence: Un pájaro puede volar.

-
Input sentence: A boat capsized.
Decoded sentence: Un te polecimo.

-
Input sentence: A car hit Tom.
Decoded sentence: Un auto atropelló a Tomás.

-
Input sentence: A coke, please.
Decoded sentence: Una Coca-Cola por favor.

-
Input sentence: A fish can swim.
Decoded sentence: Parece erepeda aquí.

-
Input sentence: A man must work.
Decoded sentence: Tono escadó.

-
Input sentence: Abandon ship!
Decoded sentence: ¡Abandonen la nave!

-
Input sentence: About what time?
Decoded sentence: ¿Apuedo Tom?

-
Input sentence: Act like a man.
Decoded sentence: Actuá como un hombre.

-
Input sentence: Act your age.
Decoded sentence: Actúa acorde a tu edad.

-
Input sentence: Add more water.
Decoded sentence: Añade más agua.

-
Input sentence: After you.
Decoded sentence: Después de usted.

-
Input sentence: Aim higher.
Decoded sentence: Más alto.

-
Input sentence: Aim. Fire!
Decoded sentence: Apunta. ¡Fuego!

-
Input s